In [1]:
from morph_utils import*
import scipy.spatial

s = 0.5

I_1, I_2, dim_1 = load_mona_lisas()

f_0 = np.array([
    [ 0, 0, 1 ],
    [ dim_1[0], 0, 1 ],
    [ 0, dim_1[1], 1 ],
    [ dim_1[0], dim_1[1], 1 ],
])
f_1, f_2 = find_face(I_1), find_face(I_2)

F = get_fundamental(f_1, f_2)

f_1 = np.vstack([ f_1, f_0 ])
f_2 = np.vstack([ f_2, f_0 ])

delu = scipy.spatial.Delaunay(f_1[:,:2]).simplices

H_1, H_2, dim_2 = get_framed_homographies(F, f_1, f_2, dim_1)

Hi_1 = np.linalg.inv(H_1)
Hi_2 = np.linalg.inv(H_2)

I_1 = cv.warpPerspective(I_1, Hi_1, dim_2)
I_2 = cv.warpPerspective(I_2, Hi_2, dim_2)
f_1 = apply_perspective(Hi_1, f_1)
f_2 = apply_perspective(Hi_2, f_2)
f_s = s * f_1 + (1 - s) * f_2
H_s = cv.getPerspectiveTransform(f_s[-4:,:2].astype(np.float32), f_0[:,:2].astype(np.float32))

tset_1 = f_1[delu][:, :, :2]
tset_2 = f_2[delu][:, :, :2]
tset_s = f_s[delu][:, :, :2]

In [2]:
I_s = 0 * I_1.copy()

for t_1, t_2, t_s, t_i in zip(tset_1, tset_2, tset_s, delu):
    if np.any(t_i > 67):
        continue

    S_1 = cv.getAffineTransform(t_1.astype(np.float32), t_s.astype(np.float32))
    S_2 = cv.getAffineTransform(t_2.astype(np.float32), t_s.astype(np.float32))

    mask = cv.bitwise_and(
        cv.drawContours(0 * I_1.copy(), [t_s.astype(np.int64)], -1, (255, 255, 255), -1),
        cv.bitwise_not(I_s)
    )

    mix = cv.addWeighted(
        cv.warpAffine(I_1.copy(), S_1, dim_2), s,
        cv.warpAffine(I_2.copy(), S_2, dim_2), 1 - s,
        0
    )

    I_s += cv.bitwise_and(mask, mix)



In [3]:
I_s = cv.warpPerspective(I_s, H_s, dim_1)
cv.imwrite('s.jpg', I_s)
cv.imwrite('1.jpg', I_1)
cv.imwrite('2.jpg', I_2)

True